In [1]:
# import needed packages
from gensim.summarization import keywords
from nltk.corpus import wordnet as wn
import requests
import time
import pandas as pd
import gensim.downloader as api
import pickle
import numpy as np

In [6]:
# load text to be prcessed. I used text from different sources to test results

# f = open("test.txt", "r")
# text = f.read()
text = "The Wandering Earth, described as China’s first big-budget science fiction thriller, quietly made it onto screens at AMC theaters in North America this weekend, and it shows a new side of Chinese filmmaking — one focused toward futuristic spectacles rather than China’s traditionally grand, massive historical epics. At the same time, The Wandering Earth feels like a throwback to a few familiar eras of American filmmaking. While the film’s cast, setting, and tone are all Chinese, longtime science fiction fans are going to see a lot on the screen that reminds them of other movies, for better or worse."


In [5]:
# this function includes all the steps involved in processing text
def process_text(text):
# first using gensim's keywords to extract most important words from text. The size of outcome can be controled by 
# the ratio argument
    kw = keywords(text, ratio=0.1, split=False, scores=False, pos_filter=None, deacc=True).split()
    lms = []
# next step is using nltk's wordnet to lemmatize extracted words
    for word in kw:
        lemma = wn.morphy(word)
        if lemma is None:
            lms.append(word)
        else:
            lms.append(lemma)
# here I am using a word2vec model that was trained on emmbeddings pretrained on wikipedia corpus
# model is used here to add words similar to the extracted keywords to increase vocabulary
    model = pickle.load(open('vectors.pkl', 'rb'))
    related_words = {}
    for w in lms:
        try:
            words = [w, model.most_similar(w, topn=4)[0][0], model.most_similar(w, topn=4)[1][0], 
                     model.most_similar(w, topn=4)[2][0], model.most_similar(w, topn=4)[3][0]]
            related_words.update({w: words})
        except:
            pass
# after that it will send queries to Yandex API to get translations for all the words we have and compile them into
# a dataframe
    new = pd.DataFrame()
    for key in related_words:
        list_ = []
        for value in related_words[key]:
            try:
                params = {'key':'dict.1.1.20190811T153301Z.6c88b4d06dce90ee.7ef3e4ec8ade64d1b8999ef0b5d1c6667df87c9f', 
                          'lang': 'en-de', 'text':value}
                response = requests.get("https://dictionary.yandex.net/api/v1/dicservice.json/lookup?", params=params)
                hmm = tuple([response.json()['def'][0]['text'], response.json()['def'][0]['tr'][0]['text']])
                list_.extend([hmm])
            except:
                pass
                list_.extend([None])
        new[key] = list_
#finally preparing the output to the user. since not all words find a corresponding translation, I opted for desplaying
# only 2 of the 4 related words to avoid showing none values in the outcome, besides the word from text
    results = []
    for row in new.T.to_numpy():
        results.append([cell for cell in row if cell is not None])
        final_table = pd.DataFrame(results)[[0,1,2]]
    return final_table

In [8]:
find_table = process_text(text)

In [9]:
# and this is a sample of how the out come would look like. The longer the text and higher ratio in the keywords
# function the more the words we are getting back will be
find_table

,0,1,2
0,"(earth, Erde)","(planet, Planet)","(Mars, Mars)"
1,"(science, Wissenschaft)","(Sciences, Biowissenschaften)","(physics, Physik)"
2,"(fiction, Fiktion)","(novels, Liebesromane)","(novel, Roman)"
